<a href="https://colab.research.google.com/github/pavi-ninjaac/DeepLearningProjects/blob/main/rock_paper_seacer_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
#keras 
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import load_model
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix

#keras layers
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,AveragePooling2D
from keras.layers import Flatten,Dropout,Dense

In [52]:
#read the dataset
def generate_dataset(folder_name):
  """
  :param folder_name: list of folder names to be considered for converting.

  :return numpy array of data and target.
  """
  data, target = [],[]
  class_count = 0
  # generate the img array.
  for name in folder_name:
    folder_path = f'/content/drive/MyDrive/Colab Notebooks/dataFiles/rock-paper-seacer/{name}'
    print(f"Collecting images from the {name}..........")
    
    for image_name in os.listdir(folder_path):
      
      img = cv2.imread(f"{folder_path}/{image_name}")
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      img = cv2.resize(img, (100, 100))
      data.append(img)
      target.append(class_count)


    class_count += 1
    print(f"Done :) , Collecting images from the {name}..........")
  
  # convert the array into numby array.
  print("Converting to numpy array........")
  data = np.array(data)
  target = np.array(target)

  return data, target
    
  
folder_name = ['rock', 'paper', 'scissors']
data_array, target_array = generate_dataset(folder_name)


Done :) , Collecting images from the rock..........
Done :) , Collecting images from the paper..........
Done :) , Collecting images from the scissors..........
Converting to numpy array........


In [53]:
data_array.shape

(1499, 100, 100)

In [26]:
a= np.array([[1,2,3,],[1,3,4]])
b = np.array(['a','b'])
c=[]
for i,ii in zip(a,b):
  c.append([i,ii])
c= np.array(c)
print(c.shape)
i = c[:,0]
i

(2, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([array([1, 2, 3]), array([1, 3, 4])], dtype=object)

In [62]:
def preprocess_dataset(data, target):
  X_train,y_train,X_test,y_test,X_val,y_val = [],[],[],[],[],[] 

  print("Shuffling the dataset...............")
  dataset = []
  for d, t  in zip(data,target):
    dataset.append([d,t])
  dataset = np.array(dataset)
  np.random.shuffle(dataset)  

  print("spliting all data...................")
  train_count, test_count, val_count = int(dataset.shape[0] * 8/10), int(dataset.shape[0] * 1/10), int(dataset.shape[0] * 1/10)
  print(train_count, test_count, val_count)

  # X_train, y_train = dataset[0:train_count+1, 0], dataset[0:train_count+1, 1]
  # X_test, y_test = dataset[train_count+1:train_count+test_count+1 , 0], dataset[train_count+1:train_count+test_count+1 , 1]
  # X_val, y_val = dataset[train_count+test_count+1:-1 , 0], dataset[train_count+test_count+1:-1 , 1]
  for i in range(dataset.shape[0]):
    d = dataset[i]
    value, target = d[0], d[1]
    
    if i <= train_count:
      X_train.append(value)
      y_train.append(target)
    elif i > train_count and i <= (train_count+test_count):
      X_test.append(value)
      y_test.append(target)
    else:
      X_val.append(value)
      y_val.append(target)
      
  print("Converting to numpy array>>>>>>>>>>>>>>>>>>")
  #convert list to numpy array
  X_train = np.array(X_train,'float32')  
  y_train = np.array(y_train,'float32')  
  X_test = np.array(X_test,'float32')  
  y_test = np.array(y_test,'float32')
  X_val = np.array(X_val,'float32')  
  y_val = np.array(y_val,'float32')
  
  print("The size of the train data-------------------->",X_train.shape)
  print("The size of the train target data------------->",y_train.shape)
  print()
  print("The size of the test data--------------------->",X_test.shape)
  print("The size of the test target data-------------->",y_test.shape)
  print()
  print("The size of the validation data--------------->",X_val.shape)
  print("The size of the validation target data-------->",y_val.shape)

  print("Normalizing the data>>>>>>>>>>>>>>>>>>>>>>")
  #normalize the data
  X_train  = X_train/255.0
  X_test = X_test/255.0
  X_val = X_val/255.0

  print("Converting target to one hot encoded values>>>>>>>>>>>>>>>>>>>>>>")
  #convert to numerical values to 0,1
  y_train = to_categorical(y_train,num_classes=3)
  y_test = to_categorical(y_test,num_classes=3)
  y_val = to_categorical(y_val,num_classes=3)

  print(X_train[0].shape)
  print("reshaping the data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
  
  X_train = X_train.reshape(X_train.shape[0] , 100 ,100 , 1)
  X_test = X_test.reshape(X_test.shape[0] , 100 ,100 , 1)
  X_val = X_val.reshape(X_val.shape[0] , 100 ,100 , 1)
  
  print("Preprocessing  completed!!!!!!!!!! stay happy :)")
  return X_train,y_train,X_test,y_test,X_val,y_val

In [64]:
X_train,y_train,X_test,y_test,X_val,y_val = preprocess_dataset(data_array, target_array)

Shuffling the dataset...............
spliting all data...................
1199 149 149
Converting to numpy array>>>>>>>>>>>>>>>>>>
The size of the train data--------------------> (1200, 100, 100)
The size of the train target data-------------> (1200,)

The size of the test data---------------------> (149, 100, 100)
The size of the test target data--------------> (149,)

The size of the validation data---------------> (150, 100, 100)
The size of the validation target data--------> (150,)
Normalizing the data>>>>>>>>>>>>>>>>>>>>>>
Converting target to one hot encoded values>>>>>>>>>>>>>>>>>>>>>>
(100, 100)
reshaping the data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Preprocessing  completed!!!!!!!!!! stay happy :)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [65]:
print("The size of the train data-------------------->",X_train.shape)
print("The size of the train target data------------->",y_train.shape)
print()
print("The size of the test data--------------------->",X_test.shape)
print("The size of the test target data-------------->",y_test.shape)
print()
print("The size of the validation data--------------->",X_val.shape)
print("The size of the validation target data-------->",y_val.shape)

The size of the train data--------------------> (1200, 100, 100, 1)
The size of the train target data-------------> (1200, 3)

The size of the test data---------------------> (149, 100, 100, 1)
The size of the test target data--------------> (149, 3)

The size of the validation data---------------> (150, 100, 100, 1)
The size of the validation target data--------> (150, 3)


# Build the model:

In [66]:
#building the model
def build_model(input_shape , num_classes , ):
  model = Sequential()
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=input_shape)) 
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same', ))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))

  #model.add(Conv2D(64, (5, 5), activation='relu', padding='same', ))
  #model.add(BatchNormalization())
  #model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Conv2D(128, (3, 3), activation='relu',  padding='same', ))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same',))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Conv2D(256, (3, 3), activation='relu', padding='same',))
  model.add(Conv2D(256, (3, 3), activation='relu',  padding='same',))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Flatten())

  model.add(Dense(1024, activation='relu',))
  model.add(Dropout(0.2))
  model.add(Dense(1024, activation='relu', ))
  model.add(Dropout(0.2))
  model.add(Dense(num_classes, activation='sigmoid'))


  # compile model
  
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  #model summary
  print(model.summary())

  return model

In [67]:
model = build_model(input_shape = (100,100,1), num_classes = 3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 64)      1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      102464    
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 128)       5

In [ ]:
# set callbacks
early_stoppping = EarlyStopping(monitor = 'val_loss',
                                min_delta = 0.001,
                                patience = 10,
                                restore_best_weights=True)

#set the global values
epoches = 50
batch_size = 64

#fit the model
history = model.fit(X_train, y_train,  
          batch_size=batch_size,  
          epochs=epoches,  
          verbose=1,  
          validation_data=(X_val, y_val),  
          shuffle=True)

Epoch 1/50
19/19 [==============================] - 335s 18s/step - loss: 3.7862 - accuracy: 0.8442 - val_loss: 27.6131 - val_accuracy: 0.4133
Epoch 2/50
19/19 [==============================] - 333s 18s/step - loss: 0.1324 - accuracy: 0.9900 - val_loss: 31.5663 - val_accuracy: 0.2867
Epoch 3/50
19/19 [==============================] - 333s 18s/step - loss: 0.1786 - accuracy: 0.9925 - val_loss: 14.1652 - val_accuracy: 0.4133
Epoch 4/50
19/19 [==============================] - 333s 18s/step - loss: 0.0485 - accuracy: 0.9975 - val_loss: 24.3523 - val_accuracy: 0.3333
Epoch 5/50
19/19 [==============================] - 333s 18s/step - loss: 0.0118 - accuracy: 0.9975 - val_loss: 11.3262 - val_accuracy: 0.4867
Epoch 6/50
19/19 [==============================] - 331s 17s/step - loss: 0.0403 - accuracy: 0.9975 - val_loss: 64.0520 - val_accuracy: 0.3267
Epoch 7/50
19/19 [==============================] - 331s 17s/step - loss: 0.1629 - accuracy: 0.9967 - val_loss: 24.5464 - val_accuracy: 0.3533